In [1]:
!pip install PyGreSQL
!pip install faker
import random
import numpy as np
from faker import Faker
import pandas as pd
import os
import csv
import pgdb
import pickle
import zipfile
import scipy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputClassifier
fake = Faker()


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
num_providers = 1000

rendering_df = pd.DataFrame()
rendering_df["RenderingProviderEntityType"] = np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
rendering_df['RenderingProviderFName'] = [fake.first_name() for i in range(num_providers)]
rendering_df['RenderingProviderMName'] = [fake.first_name() for i in range(num_providers)]
rendering_df['RenderingProviderLName'] = [fake.last_name() for i in range(num_providers)]
rendering_df['RenderingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_providers)]
rendering_df['RenderingProviderTIN']= np.random.randint(100000000,999999999, size=num_providers)
rendering_df["RenderingProviderID"] = np.random.randint(10000,999999, size=num_providers)
rendering_df["RenderingProviderID"] = rendering_df["RenderingProviderID"].apply(str)
rendering_df['1'] = [fake.building_number() for i in range(num_providers)]
rendering_df["2"] = [fake.street_name() for i in range(num_providers)]
rendering_df['RenderingProviderAddressLine1'] = rendering_df["1"]+" "+rendering_df["2"]
rendering_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.1, 0.3, 0.6])
rendering_df["2"] = [fake.building_number() for i in range(num_providers)]
rendering_df["RenderingProviderAddressLine2"] = rendering_df["1"]+" "+rendering_df["2"]
rendering_df.loc[rendering_df['RenderingProviderAddressLine2'].str.contains('Null', case=False), 'RenderingProviderAddressLine2'] = 'Null'
rendering_df = rendering_df.drop(columns=['1',"2"])
rendering_df['RenderingProviderCity'] = [fake.city() for i in range(num_providers)]
rendering_df['RenderingProviderOrg']= np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
rendering_df['RenderingProviderPhoneNumber'] = rendering_df['RenderingProviderPhoneNumber'].apply(str)
rendering_df['RenderingProviderTIN'] = rendering_df['RenderingProviderTIN'].apply(str)
rendering_df["CCN"] = np.random.randint(0,38000, size=num_providers)
rendering_df["CCN"] = rendering_df["CCN"].apply(lambda x: '{0:0>5}'.format(x))
rendering_df["RenderingProviderNPI"] = np.random.randint(10000,999999, size=num_providers)
rendering_df["RenderingProviderNPI"] = rendering_df["RenderingProviderNPI"].apply(str)
rendering_df['ren_id'] = range(1, 1+len(rendering_df))
#ren = zip_df["renderingproviderzipcode"].tolist()
#rendering_df["zip"] = np.random.choice(list(zip_df["renderingproviderzipcode"].unique()), size = num_providers)

rendering_df


,RenderingProviderEntityType,RenderingProviderFName,RenderingProviderMName,RenderingProviderLName,RenderingProviderPhoneNumber,RenderingProviderTIN,RenderingProviderID,RenderingProviderAddressLine1,RenderingProviderAddressLine2,RenderingProviderCity,RenderingProviderOrg,CCN,RenderingProviderNPI,ren_id
0,O,Danny,Rhonda,Johnson,8971854781703,928942751,695056,4006 Yolanda Summit,Null,Alisonberg,O,10701,75076,1
1,O,Gregory,Emily,Dean,1114839947975,529890822,205009,1708 Cody Station,Apt. 8104,West Jamesland,O,34282,427945,2
2,O,William,Daniel,Hunt,8976198687737,371181725,386300,521 Powell View,Null,Greenchester,I,34356,424176,3
3,I,John,Christopher,Sellers,0075870576572,130003856,123027,1938 Jessica Motorway,Apt. 0146,Heatherchester,O,11065,736645,4
4,I,William,Steven,Graham,1075199945186,613353317,893877,92643 Marshall Plaza,Null,Port Brucestad,O,30784,199986,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,O,Margaret,Susan,Lewis,1089902125294,168761718,435108,7764 Mark Burg,Null,New Keithchester,I,03060,358466,996
996,O,Linda,Jessica,Smith,8166606097793,887345881,84284,31894 Perry Prairie,Apt. 1508,Jasontown,O,25269,582531,997
997,I,Christopher,Kenneth,Hamilton,0962613538923,292050703,626085,3560 Mccarthy Way,Null,West Evanfort,I,26936,496107,998
998,O,Brandon,Tina,Barnes,7899985694612,701997508,932746,487 Wood Ford,Apt. 6649,West Paul,O,19983,293595,999


In [3]:
#billing_df#add zip just like for rendering prov



In [4]:
#Probability of number of claims per provider
#We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.
#Based on MonteCarlo simulation from the empirical data
num_claims_df = pd.read_csv('num_claims_df.csv')

#How many providers do we want to synthesize?
## User defined for each specialty
num_providers = 1000
num_iter = 100000
num_claims_avg_lst = []
for ii in range(0, num_iter):
    num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
dist_mean = np.round(np.mean(num_claims_avg_lst))
dist_std = np.round(np.std(num_claims_avg_lst))
group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
scipy.stats.describe(group)

### Determine probability of different combinations 
### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
all_combinations = pd.read_csv('all_combinations_age.csv')
sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]
sel_combinations

,combos,place_of_service,pat_gender,age_bin,diag_values,num_claims,prob_claims
6052,90791;GT@WD,22,F,Adult,F411,886,0.010072
10067,90791;WD,30,F,Adult,F411,1269,0.014425
10198,90791;WD,30,F,Adult,F4323,1368,0.015551
12231,90791;WD,30,None,Adult,F331,1201,0.013652
12630,90791;WD,30,None,Adult,F411,3007,0.034182
...,...,...,...,...,...,...,...
71273,90837;WD:90837;WD,30,None,Adult,F4322,1029,0.011697
71276,90837;WD:90837;WD,30,None,Adult,F4323,1840,0.020916
80826,90847;WD,30,None,Adult,F4323,1863,0.021178
82417,90853;WC,30,None,Adult,F1120,1476,0.016778


In [5]:
## Unique combinations and corresponding weights
comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
weights = sel_combinations['prob_claims'].tolist()

final_df_list = []
random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
for ii in range(0, num_providers):
    if ii in random_outliers:
        tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    else:
        tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    tmp['ren_id'] = str(ii+1)
    tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
    final_df_list.append(tmp)
final_df = pd.concat(final_df_list)

a = []#wsplitting up combo into lines, in list
for index, row in final_df.iterrows():  
    b = (row['combos'].split(":"))
    a.append(b)
final_df["combos"] = a
final_df['ClaimID'] = range(1, 1+len(final_df))

final_df

,combos,place_of_service,pat_gender,age_bin,diag_values,ren_id,prov_claim_id,ClaimID
0,[90837;95@WD],22,F,Adult,F4323,1,claim_0,1
1,[90837;95@WD],22,F,Adult,F411,1,claim_1,2
2,[90837;WD],30,F,Adult,F411,1,claim_2,3
3,[90837;WD],30,None,Adult,F4320,1,claim_3,4
4,[90791;WD],30,None,Adult,F4322,1,claim_4,5
...,...,...,...,...,...,...,...,...
120,[90837;WD],30,F,Adult,F330,1000,claim_120,134898
121,[90837;GT@WD],22,F,Adult,F419,1000,claim_121,134899
122,[90837;95@WD],22,F,Adult,F4320,1000,claim_122,134900
123,[90837;WD],30,F,Minor,F4322,1000,claim_123,134901


In [6]:
num_prov = 1000


In [7]:

pat_df = pd.DataFrame()

pat_df["PatientState"] = [fake.state() for i in range(num_prov)]
pat_df['PatientFName'] = [fake.first_name() for i in range(num_prov)]
#######
pat_df["pat_gender"] = np.random.choice(["Male", "Female", "Other"], pat_df.shape[0], p=[0.49, 0.49, 0.02])
pat_df["age_bin"] = np.random.choice(["Adult", "Elderly", "Minor"], pat_df.shape[0], p=[0.4, 0.4, 0.2])
######
pat_df["PatientLName"] = [fake.last_name() for i in range(num_prov)]
pat_df['PatientMName'] = [fake.first_name() for i in range(num_prov)]
pat_df["PatientID"] = np.random.randint(1000000000, 9999999999, size=num_prov)
pat_df["PatientID"] = pat_df["PatientID"].astype(str)
pat_df['1'] = [fake.building_number() for i in range(num_prov)]
pat_df["2"] = [fake.street_name() for i in range(num_prov)]
pat_df['PatientAddressLine1'] = pat_df["1"]+" "+pat_df["2"]
pat_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_prov, p=[0.1, 0.3, 0.6])
pat_df["2"] = [fake.building_number() for i in range(num_prov)]
pat_df["PatientAddressLine2"] = pat_df["1"]+" "+pat_df["2"]
pat_df.loc[pat_df['PatientAddressLine2'].str.contains('Null', case=False), 'PatientAddressLine2'] = 'Null'
pat_df['PatientCity'] = [fake.city() for i in range(num_prov)]
pat_df["SubscriberState"]= pat_df["PatientState"]
pat_df["SubscriberID"]= np.random.randint(1000000000, 9999999999, size=num_prov)
pat_df["SubscriberID"]=pat_df["SubscriberID"].astype(str)
pat_df["PatientRelationship"] = np.random.choice(["Dependant", "Spouse", "Subscriber"], pat_df.shape[0], p=[0.2, 0.4, 0.4])


adult_df = pat_df.loc[(pat_df['age_bin'] == 'Adult')]
elderly_df = pat_df.loc[(pat_df['age_bin'] == 'Elderly')]
minor_df = pat_df.loc[(pat_df['age_bin'] == 'Minor')]

adult_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 22, maximum_age=65) for i in range(adult_df.shape[0])]
elderly_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 65, maximum_age=99) for i in range(elderly_df.shape[0])]
minor_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 1, maximum_age=18) for i in range(minor_df.shape[0])]

adult_df["PatientRelationship"] = np.random.choice(["Dependant", "Spouse", "Subscriber"], adult_df.shape[0], p=[0.2, 0.4, 0.4])
elderly_df["PatientRelationship"] = np.random.choice(["Spouse", "Subscriber"], elderly_df.shape[0], p=[0.5, 0.5])
minor_df["PatientRelationship"] = "Dependant"
pat_df = pd.concat([adult_df, elderly_df, minor_df])


Sub_df = pat_df.loc[(pat_df['PatientRelationship'] == 'Subscriber')]
NS_df = pat_df.loc[(pat_df['PatientRelationship'] != 'Subscriber')]

NS_df["SubscriberGender"] = np.random.choice(["Male", "Female", "Other"], NS_df.shape[0], p=[0.49, 0.49, 0.02])
NS_df['SubscriberDOB'] = [fake.date_of_birth(minimum_age= 22, maximum_age=99) for i in range(NS_df.shape[0])]
NS_df['SubscriberCity'] = [fake.city() for i in range(NS_df.shape[0])]

NS_df['1'] = [fake.building_number() for i in range(NS_df.shape[0])]
NS_df["2"] = [fake.street_name() for i in range(NS_df.shape[0])]
NS_df['SubscriberAddressLine1'] = NS_df["1"]+" "+NS_df["2"]
NS_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], NS_df.shape[0], p=[0.1, 0.3, 0.6])
NS_df["2"] = [fake.building_number() for i in range(NS_df.shape[0])]
NS_df["SubscriberAddressLine2"] = NS_df["1"]+" "+NS_df["2"]
NS_df.loc[NS_df['SubscriberAddressLine2'].str.contains('Null', case=False), 'SubscriberAddressLine2'] = 'Null'
NS_df['SubscriberLName'] = NS_df['PatientLName']
NS_df['SubscriberFName'] = [fake.first_name() for i in range(NS_df.shape[0])]
NS_df['SubscriberMName'] = [fake.first_name() for i in range(NS_df.shape[0])]
NS_df = NS_df.drop(columns = ["1","2"])
#####

Sub_df["SubscriberGender"] = Sub_df["pat_gender"]
Sub_df['SubscriberDOB'] = Sub_df['PatientDOB']
Sub_df['SubscriberCity'] = Sub_df['PatientCity']
Sub_df['SubscriberAddressLine1'] = Sub_df['PatientAddressLine1']
Sub_df["SubscriberAddressLine2"] = Sub_df["PatientAddressLine2"]
Sub_df['SubscriberLName'] = Sub_df['PatientLName']
Sub_df['SubscriberFName'] = Sub_df['PatientFName']
Sub_df['SubscriberMName'] = Sub_df['PatientMName']

pat_df = pd.concat([Sub_df, NS_df])
pat_df = pat_df.drop(columns=['1',"2"])

pat_df["PatientDOB"]=pat_df["PatientDOB"].astype(str)
pat_df["SubscriberDOB"]=pat_df["SubscriberDOB"].astype(str)
    
    
    
adult_df = pat_df.loc[(pat_df['age_bin'] == 'Adult')]
elderly_df = pat_df.loc[(pat_df['age_bin'] == 'Elderly')]
minor_df = pat_df.loc[(pat_df['age_bin'] == 'Minor')]


adult_df["Pat"]= adult_df['PatientState']+"@"+adult_df['PatientFName']+"@"+adult_df['pat_gender']+"@"+adult_df['PatientLName']+"@"+adult_df['PatientMName']+"@"+adult_df['PatientID']+"@"+adult_df['PatientAddressLine1']+"@"+adult_df['PatientAddressLine2']+"@"+adult_df['PatientCity']+"@"+adult_df['SubscriberState']+"@"+adult_df['SubscriberID']+"@"+adult_df['PatientRelationship']+"@"+adult_df['PatientDOB']+"@"+adult_df['SubscriberGender']+"@"+adult_df['SubscriberDOB']+"@"+adult_df['SubscriberCity']+"@"+adult_df['SubscriberAddressLine1']+"@"+adult_df['SubscriberAddressLine2']+"@"+adult_df['SubscriberLName']+"@"+adult_df['SubscriberFName']+"@"+adult_df['SubscriberMName']
adult_list = adult_df["Pat"].tolist()


elderly_df["Pat"]= elderly_df['PatientState']+"@"+elderly_df['PatientFName']+"@"+elderly_df['pat_gender']+"@"+elderly_df['PatientLName']+"@"+elderly_df['PatientMName']+"@"+elderly_df['PatientID']+"@"+elderly_df['PatientAddressLine1']+"@"+elderly_df['PatientAddressLine2']+"@"+elderly_df['PatientCity']+"@"+elderly_df['SubscriberState']+"@"+elderly_df['SubscriberID']+"@"+elderly_df['PatientRelationship']+"@"+elderly_df['PatientDOB']+"@"+elderly_df['SubscriberGender']+"@"+elderly_df['SubscriberDOB']+"@"+elderly_df['SubscriberCity']+"@"+elderly_df['SubscriberAddressLine1']+"@"+elderly_df['SubscriberAddressLine2']+"@"+elderly_df['SubscriberLName']+"@"+elderly_df['SubscriberFName']+"@"+elderly_df['SubscriberMName']
elderly_list = elderly_df["Pat"].tolist()

minor_df["Pat"]= minor_df['PatientState']+"@"+minor_df['PatientFName']+"@"+minor_df['pat_gender']+"@"+minor_df['PatientLName']+"@"+minor_df['PatientMName']+"@"+minor_df['PatientID']+"@"+minor_df['PatientAddressLine1']+"@"+minor_df['PatientAddressLine2']+"@"+minor_df['PatientCity']+"@"+minor_df['SubscriberState']+"@"+minor_df['SubscriberID']+"@"+minor_df['PatientRelationship']+"@"+minor_df['PatientDOB']+"@"+minor_df['SubscriberGender']+"@"+minor_df['SubscriberDOB']+"@"+minor_df['SubscriberCity']+"@"+minor_df['SubscriberAddressLine1']+"@"+minor_df['SubscriberAddressLine2']+"@"+minor_df['SubscriberLName']+"@"+minor_df['SubscriberFName']+"@"+minor_df['SubscriberMName']
minor_list = minor_df["Pat"].tolist()


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

zip_df =pd.read_csv("location_data.csv")
zip_combinations = zip_df[zip_df.prob_combo>=0.0001]
group = [1]*final_df.shape[0]

comb_list = zip_combinations.drop(['prob_combo'], axis = 1).values
weights = zip_combinations['prob_combo'].tolist()

final_df_list = []
zip_length = rendering_df.shape[0] + 10
for ii in range(0, zip_length):
    
    final_df_list.append(tmp)

zip_df =pd.read_csv("location_data.csv")
zip_combinations = zip_df[zip_df.prob_combo>=0.0001]

comb_list = zip_combinations.drop(['prob_combo'], axis = 1).values
weights = zip_combinations['prob_combo'].tolist()

zip_df = pd.DataFrame(random.choices(comb_list, weights,
                                     k=zip_length), columns = zip_df.drop(['prob_combo'], axis = 1).columns)

rend_zips = zip_df['renderingproviderzipcode'].tolist()
rendering_df['renderingproviderzipcode'] = np.random.choice(rend_zips, rendering_df.shape[0])
rendering_df

zip_df[zip_df['renderingproviderzipcode'] == '5602'].drop_duplicates()



rend_zip_df = rendering_df.merge(zip_df.drop_duplicates(), on='renderingproviderzipcode')


rend_zip_df[rend_zip_df['RenderingProviderID'] == '921526']['renderingproviderzipcode']



rend_zip_df.values
rend_tin = rend_zip_df['RenderingProviderTIN'].unique()


dfRendChoices = pd.DataFrame()
rend_tin = rend_zip_df['RenderingProviderTIN'].unique()
i = 0
for tin in rend_tin:
    comb_list = rend_zip_df[rend_zip_df['RenderingProviderTIN'] == tin].values
    random.choices([1, 2, 3], k=1)
    
    dfTmp = pd.DataFrame(random.choices(comb_list, k=random.choices([1, 2, 3,4 5], k=1)[0]), columns = rend_zip_df.columns)
    #dfTmp["ID"]= i
    i+=1
    dfRendChoices = dfRendChoices.append(dfTmp)
dfRendChoices = dfRendChoices.drop_duplicates()

In [8]:
#dfRendChoices

In [9]:
#dfTmp

In [10]:

num_lines= final_df.shape[0]
taxonomy = 25#may want to look at this more /not right
"""
zip_df =pd.read_csv("location_data.csv")
zip_combinations = zip_df[zip_df.prob_combo>=0.0001]
group = [1]*final_df.shape[0]

comb_list = zip_combinations.drop(['prob_combo'], axis = 1).values
weights = zip_combinations['prob_combo'].tolist()

final_df_list = []
for ii in range(0, final_df.shape[0]):
    tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), columns = zip_df.drop(['prob_combo'], axis = 1).columns)
    final_df_list.append(tmp)
    
zip_df = pd.concat(final_df_list)
zip_df["billingproviderzipcode"] = zip_df["billingproviderzipcode"].astype(float)
zip_df["billingproviderzipcode"] = zip_df["billingproviderzipcode"].astype(int)
zip_df["pickupzipcode"] = zip_df["pickupzipcode"].astype(float)
zip_df["pickupzipcode"] = zip_df["pickupzipcode"].astype(int)
zip_df["dropoffzipcode"] = zip_df["dropoffzipcode"].astype(float)
zip_df["dropoffzipcode"] = zip_df["dropoffzipcode"].astype(int)
zip_df["renderingproviderzipcode"] = zip_df["renderingproviderzipcode"].astype(float)
zip_df["renderingproviderzipcode"] = zip_df["renderingproviderzipcode"].astype(int)
zip_df["prescribingproviderzipcode"] = zip_df["prescribingproviderzipcode"].astype(float)
zip_df["prescribingproviderzipcode"] = zip_df["prescribingproviderzipcode"].astype(int)
zip_df["pickupzipcode"]=zip_df["pickupzipcode"].astype(str)
zip_df["dropoffzipcode"]=zip_df["dropoffzipcode"].astype(str)
zip_df["billingproviderzipcode"]=zip_df["billingproviderzipcode"].astype(str)
zip_df['renderingproviderzipcode']=zip_df['renderingproviderzipcode'].astype(str)
zip_df['prescribingproviderzipcode']=zip_df['prescribingproviderzipcode'].astype(str)
zip_df['subscriberzipcode']=zip_df['subscriberzipcode'].astype(str)
zip_df['patientzipcode']=zip_df['patientzipcode'].astype(str)
zip_df['billingproviderstate']=zip_df['billingproviderstate'].astype(str)
zip_df["renderingproviderstate"]=zip_df["renderingproviderstate"].astype(str)
zip_df['subscriberstate']=zip_df['subscriberstate'].astype(str)
zip_df['prescribingproviderstate']=zip_df['prescribingproviderstate'].astype(str)

zip_df["zips"] = zip_df["pickupzipcode"]+"@"+zip_df["dropoffzipcode"]+"@"+zip_df["billingproviderzipcode"]+"@"+ zip_df['renderingproviderzipcode']+"@"+zip_df['prescribingproviderzipcode']+"@"+zip_df['subscriberzipcode']+"@"+zip_df['patientzipcode'] +"@"+ zip_df['billingproviderstate']+"@"+zip_df["renderingproviderstate"]+"@"+zip_df['subscriberstate']+"@"+zip_df['prescribingproviderstate']
zip_list = zip_df["zips"].tolist()
print("zipped")"""



billing_df = pd.DataFrame()
billing_df["BillingProviderID"] = np.random.randint(10000,999999, size=num_prov)
billing_df["BillingProviderTIN"] = np.random.randint(100000000,999999999, size=num_prov)
billing_df["BillingProviderOrg"] = np.random.choice(["I", "O"], num_prov, p=[0.5, 0.5])
billing_df['BillingProviderMName'] = [fake.first_name() for i in range(num_prov)]
billing_df['BillingProviderFName'] = [fake.first_name() for i in range(num_prov)]
billing_df['BillingProviderLName'] = [fake.last_name() for i in range(num_prov)]
billing_df['BillingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_prov)]
billing_df['BillingProviderCity'] = [fake.city() for i in range(num_prov)]
billing_df["BillingProviderEntityType"] = np.random.choice(["I", "O"], num_prov, p=[0.5, 0.5])
billing_df["BillingProviderNPI"]  = np.random.randint(10000,999999, size=num_prov)
billing_df['1'] = [fake.building_number() for i in range(num_prov)]
billing_df["2"] = [fake.street_name() for i in range(num_prov)]
billing_df['BillingProviderAddressLine1'] = billing_df["1"]+" "+billing_df["2"]
billing_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_prov, p=[0.1, 0.3, 0.6])
billing_df["2"] = [fake.building_number() for i in range(num_prov)]
billing_df["BillingProviderAddressLine2"] = billing_df["1"]+" "+billing_df["2"]
billing_df.loc[billing_df['BillingProviderAddressLine2'].str.contains('Null', case=False), 'BillingProviderAddressLine2'] = 'Null'
billing_df = billing_df.drop(columns=['1',"2"])
billing_df["BillingProviderNPI"] = billing_df["BillingProviderNPI"].apply(str)
billing_df['BillingProviderID'] = billing_df['BillingProviderID'].apply(str)
billing_df['BillingProviderTIN'] = billing_df['BillingProviderTIN'].apply(str)
billing_df["Billing"] = billing_df["BillingProviderID"]+"@"+billing_df["BillingProviderTIN"]+"@"+billing_df["BillingProviderOrg"]+"@"+billing_df["BillingProviderFName"]+"@"+billing_df["BillingProviderMName"]+"@"+billing_df["BillingProviderLName"]+"@"+billing_df["BillingProviderPhoneNumber"]+"@"+billing_df["BillingProviderAddressLine1"]+"@"+billing_df["BillingProviderAddressLine2"]+"@"+billing_df["BillingProviderCity"]+"@"+billing_df["BillingProviderEntityType"]+"@"+billing_df["BillingProviderNPI"]
billing_list = billing_df["Billing"].tolist()

group_df = pd.DataFrame()
group_df['GroupName'] = [fake.company() for i in range(num_prov)]
group_df['GroupNumber']  = np.random.randint(10000,999999, size=num_prov)
group_df['GroupNumber'] = group_df['GroupNumber'].apply(str)
group_df["Group"] = group_df['GroupName']+"@"+group_df['GroupNumber']
group_list = group_df["Group"].tolist()

tooth_df = pd.DataFrame()
tooth_df["ToothNumber"] = np.random.choice(["Null"], num_prov)
tooth_df["ToothBegin"] = np.random.choice(["Null"], num_prov)
tooth_df["ToothEnd"] = np.random.choice(["Null"], num_prov)
tooth_df["ToothSurfaces"] = "Null"
tooth_df["Tooth"] = tooth_df["ToothNumber"]+"@"+tooth_df["ToothBegin"]+"@"+tooth_df["ToothEnd"]+"@"+tooth_df["ToothSurfaces"]
tooth_list = tooth_df["Tooth"].tolist()

certifying_df = pd.DataFrame()
"""
certifying_df['CertifyingPhysicianNme']= [fake.last_name() for i in range(num_prov)]
certifying_df["CertifyingPhysicianID"] = np.random.randint(10000,999999, size=num_prov)
certifying_df["CertifyingPhysicianNPI"] = np.random.randint(10000,999999, size=num_prov)
certifying_df["CertifyingPhysicianNPI"] =certifying_df["CertifyingPhysicianNPI"].apply(str)
certifying_df["CertifyingPhysicianID"] = certifying_df["CertifyingPhysicianID"].apply(str)"""

final_df['CertifyingPhysicianName'] = " Null"
final_df["CertifyingPhysicianID"]  = "Null"
final_df["CertifyingPhysicianNPI"] = "Null"
final_df["CertifyingPhysicianNPI"] ="Null"
final_df["CertifyingPhysicianID"]  = "Null"


#certifying_df["Certifying"] =certifying_df["CertifyingPhysicianNme"]+"@"+certifying_df['CertifyingPhysicianID']+"@"+certifying_df["CertifyingPhysicianNPI"]
#certifying_list = certifying_df["Certifying"].tolist()

ordering_df = pd.DataFrame()
"""ordering_df["OrderingProviderID"] = np.random.randint(10000,999999, size=num_prov)
ordering_df['OrderingProviderName'] = [fake.last_name() for i in range(num_prov)]
ordering_df["OrderingProviderID"] =ordering_df["OrderingProviderID"].apply(str)
ordering_df["OrderingProviderNPI"]  = np.random.randint(10000,999999, size=num_prov)
ordering_df["OrderingProviderNPI"] = ordering_df["OrderingProviderNPI"].apply(str)"""

final_df["OrderingProviderID"] = " Null"
final_df['OrderingProviderName'] = "Null"
final_df["OrderingProviderID"] ="Null"
final_df["OrderingProviderNPI"] ="Null"

#ordering_df["Ordering"] =ordering_df["OrderingProviderID"]+"@"+ordering_df['OrderingProviderName']+"@"+ordering_df["OrderingProviderNPI"]
#ordering_list = ordering_df["Ordering"].tolist()

attending_df = pd.DataFrame()
attending_df['AttendingProviderNAME'] = [fake.last_name() for i in range(num_prov)]
attending_df["AttendingProviderNPI"] = np.random.randint(10000,999999, size=num_prov)
attending_df["AttendingProviderNPI"] = attending_df["AttendingProviderNPI"].apply(str)
attending_df["Attending"] = attending_df['AttendingProviderNAME']+"@"+attending_df["AttendingProviderNPI"]
attending_list = attending_df["Attending"].tolist()

prescribing_df = pd.DataFrame()
"""prescribing_df["PrescribingProviderID"] = np.random.randint(10000,999999, size=num_prov)
prescribing_df["PrescribingProviderTIN"] = np.random.randint(100000000,999999999, size=num_prov)
prescribing_df["PrescribingProviderOrg"] = np.random.choice(["I", "O"], num_prov, p=[0.5, 0.5])
prescribing_df['PrescribingProviderFName'] = [fake.first_name() for i in range(num_prov)]
prescribing_df['PrescribingProviderMName'] = [fake.first_name() for i in range(num_prov)]
prescribing_df['PrescribingProviderLName'] = [fake.last_name() for i in range(num_prov)]
prescribing_df['PrescribingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_prov)]
prescribing_df['1'] = [fake.building_number() for i in range(num_prov)]
prescribing_df["2"] = [fake.street_name() for i in range(num_prov)]
prescribing_df['PrescribingProviderAddressLine1'] = prescribing_df["1"]+" "+prescribing_df["2"]
prescribing_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_prov, p=[0.1, 0.3, 0.6])
prescribing_df["2"] = [fake.building_number() for i in range(num_prov)]
prescribing_df["PrescribingProviderAddressLine2"] = prescribing_df["1"]+" "+prescribing_df["2"]
prescribing_df.loc[prescribing_df['PrescribingProviderAddressLine2'].str.contains('Null', case=False), 'PrescribingProviderAddressLine2'] = 'Null'
prescribing_df['PrescribingProviderCity'] = [fake.city() for i in range(num_prov)]
prescribing_df = prescribing_df.drop(columns=['1',"2"])
prescribing_df["PrescribingProviderNPI"] = np.random.randint(10000,999999, size=num_prov)
prescribing_df['PrescribingProviderNPI'] = prescribing_df['PrescribingProviderNPI'].apply(str)
prescribing_df['PrescribingProviderID'] = prescribing_df['PrescribingProviderID'].apply(str)
prescribing_df['PrescribingProviderTIN'] = prescribing_df['PrescribingProviderTIN'].apply(str)
"""
final_df["PrescribingProviderID"] = "Null"
final_df["PrescribingProviderTIN"] = "Null"
final_df["PrescribingProviderOrg"] = "Null"
final_df['PrescribingProviderFName'] = "Null"
final_df['PrescribingProviderMName'] = "Null"
final_df['PrescribingProviderLName'] = "Null"
final_df['PrescribingProviderPhoneNumber'] = "Null"
final_df['PrescribingProviderAddressLine1'] = "Null"
final_df["PrescribingProviderAddressLine2"] = "Null"
final_df['PrescribingProviderCity'] = "Null"

final_df["PrescribingProviderNPI"] = "Null"
final_df['PrescribingProviderNPI'] = "Null"
final_df['PrescribingProviderID'] = "Null"
final_df['PrescribingProviderTIN'] = "Null"

#prescribing_df["Prescribing"] = prescribing_df["PrescribingProviderID"]+"@"+prescribing_df["PrescribingProviderTIN"]+"@"+prescribing_df["PrescribingProviderOrg"]+"@"+ prescribing_df['PrescribingProviderFName']+"@"+prescribing_df['PrescribingProviderMName']+"@"+prescribing_df['PrescribingProviderLName']+"@"+prescribing_df['PrescribingProviderPhoneNumber'] +"@"+ prescribing_df['PrescribingProviderAddressLine1']+"@"+prescribing_df["PrescribingProviderAddressLine2"]+"@"+prescribing_df['PrescribingProviderCity']+"@"+prescribing_df['PrescribingProviderNPI']
#prescribing_list = prescribing_df["Prescribing"].tolist()



#########

final_df["ClaimType"] = "P" #P for profesional, D for dental, F for facility

final_df["ContractedFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["InterPlan"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["ASOFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["PrimaryFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["OutOfNetwork"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["CapEpcIndicator"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["DenyFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.1, 0.9])

final_df["PreAuthIndicator"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
final_df["DenyFlag"] = np.random.choice(["P", "D"], num_lines, p=[0.5, 0.5])
final_df["PayeeType"] = np.random.choice(["Prov", "Subscriber"], num_lines, p=[0.9, 0.1])
final_df["LineConsideredAmount"] =  "Null"
final_df["PreAuthNumber"] = "Null"
final_df["ICDVersion"] = np.random.choice(["ICD-10","ICD-9"], num_lines, p =[0.8,0.2])
final_df["PriorClaimID"] = "Null"
final_df["LineAdjustmentReasonCode"] = "Null"
final_df["RelatedHospDischargeDate"] = "Null"
final_df["RelatedHospAdmitDate"] = "Null"
final_df["ClaimAdjustmentReasonCode"] = "Null"
final_df["NDC"] = "Null"
final_df["LineAllowedAmount"] = "Null"
final_df["DenyReason"] = "Null"
final_df["ConditionCode"] = "Null"
final_df["ValueCode"] = "Null"#only not "Null" for facility claims
final_df["ServiceUnits"] = "Null"
final_df["LineUnits"] = "Null"
final_df["LineUnitType"] = "Null"
final_df["CLIA"]="NUll"# null for all but Clinical labs
final_df["PrescribingProviderTaxonomy"]= str(taxonomy)
final_df["BillingProviderTaxonomy"]= str(taxonomy)
final_df["RenderingProviderTaxonomy"]= str(taxonomy)
final_df["PrescribingProviderSpecialtyCode"]= str(taxonomy)
final_df["BillingProviderSpecialtyCode"]= str(taxonomy)
final_df["RenderingProviderSpecialtyCode"] = str(taxonomy)

final_df["LineOfBusiness"] = np.random.choice(["Commercial", "Medicare", "Commercial Plus"], num_lines, p=[0.3, 0.4, 0.3])
final_df["InterPlan"] = np.random.choice(["ITS", "nonITS", "HOST"], num_lines, p=[0.3, 0.4, 0.3])
final_df["OccurrenceCode"] = np.random.choice(["00", "01","02","03","04","05","06","09",], num_lines, p=[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])
final_df["DRGVersion"] = np.random.choice(["37", "38","39"], num_lines, p=[0.4, 0.3,0.3])
final_df["RelatedCausesCode"] = np.random.choice(["AA", "AP","EM", "OA"], num_lines, p=[0.2, 0.1,0.3, 0.4])

final_df["ReferringProviderNPI"] = np.random.randint(10000,999999, size=num_lines)
final_df["OtherPhysicianNPI"] = np.random.randint(10000,999999, size=num_lines)
final_df["AdmitSource"]= np.random.randint(1, 9, size=num_lines)
final_df["AdmitType"] =np.random.randint(1, 9, size=num_lines)
final_df["DischargeStatus"] = np.random.randint(0, 99, size=num_lines)
final_df['OrderingProviderName'] = [fake.last_name() for i in range(num_lines)]
final_df['OtherPhysicianNAME'] = [fake.last_name() for i in range(num_lines)]

final_df["TypeofBill"]= np.random.randint(0,999, size=num_lines)
final_df["TypeofBill"]=final_df["TypeofBill"].apply(lambda x: '{0:0>3}'.format(x))
final_df["NumofAdjustments"]= np.random.randint(0,10, size=num_lines)
final_df["RevenueCode"]= np.random.randint(0,9999, size=num_lines)
final_df["RevenueCode"]=final_df["RevenueCode"].apply(lambda x: '{0:0>4}'.format(x))
final_df["TypeofBill"]= np.random.randint(0,999, size=num_lines)
final_df["TypeofBill"]=final_df["TypeofBill"].apply(lambda x: '{0:0>4}'.format(x))

final_df["Billing"] = np.random.choice(billing_list, size=num_lines)
#final_df["Prescribing"] = np.random.choice(prescribing_list, size=num_lines)#change to Null
#final_df["Ordering"] = np.random.choice(ordering_list, size=num_lines)#Null
#final_df["Certifying"] = np.random.choice(certifying_list, size=num_lines)    #Null
final_df["Tooth"] = np.random.choice(tooth_list, size=num_lines)
final_df["Group"] = np.random.choice(group_list, size=num_lines)
#final_df["Zip"] = np.random.choice(zip_list, size=num_lines)



##################
adult_df = final_df.loc[(final_df['age_bin'] == 'Adult')]
adult_df["Pat"] = np.random.choice(adult_list, size=adult_df.shape[0])

elderly_df = final_df.loc[(final_df['age_bin'] == 'Elderly')]
elderly_df["Pat"] = np.random.choice(elderly_list, size=elderly_df.shape[0])

minor_df = final_df.loc[(final_df['age_bin'] == 'Minor')]
minor_df["Pat"] = np.random.choice(minor_list, size=minor_df.shape[0])

final_df = pd.concat([minor_df, elderly_df, adult_df])


final_df["Attending"] = np.random.choice(attending_list, size=num_lines)
final_df["AccidentDate"] = np.random.choice(pd.date_range('2017-01-01', '2022-01-01'), len(final_df))
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["AdmitDate"] = final_df["AccidentDate"]+delta
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["ClaimServiceStartDate"] = final_df["AdmitDate"] +delta
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["LineServiceStartDate"] = final_df["ClaimServiceStartDate"]+delta
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["LineServiceEndDate"]=final_df["LineServiceStartDate"]  + delta
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["ClaimServiceEndDate"]=final_df["LineServiceEndDate"]+delta 
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["ClaimReceivedDate"] = final_df["ClaimServiceEndDate"]+delta
delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
final_df["ClaimPaidDate"] = final_df["ClaimReceivedDate"] +delta
final_df["LineServiceStatusCategoryCode"] = final_df["AdmitDate"]

final_df["EmergencyIndicator"] = "Null"
final_df["AdmittingDiag"]= "Null"
final_df["ICDProcDate1"]="Null"
final_df["ICDProcDate2"]="Null"
final_df["ICDProcDate3"]="Null"
final_df["ICDProcDate4"]="Null"
final_df["ICDProc1"]= "Null"
final_df["ICDProc2"]= "Null"
final_df["ICDProc3"]= "Null"
final_df["ICDProc4"]= "Null"

final_df["HIPPSCode"]= "Null"
final_df["ClaimStatusCategoryCode"] = "Null"
final_df["HCPC_Rate_HIPPSCode"]= "Null"
final_df["DRGCode"]= "Null"

"""final_df["ICDProcDate1"]=final_df["LineServiceStartDate"]     #for facility claims
final_df["ICDProcDate2"]=final_df["LineServiceStartDate"]
final_df["ICDProcDate3"]=final_df["LineServiceStartDate"]
final_df["ICDProcDate4"]=final_df["LineServiceStartDate"]"""

final_df
def splitting(name, delineator = "@"):
    a = []
    for index, row in final_df.iterrows():  
        b = (row[name].split(delineator))
        a.append(b)
    final_df[name] = a

splitting("Billing")
#splitting("Prescribing")
#splitting("Ordering")
#splitting("Certifying")
splitting("Tooth")
splitting("Group")
splitting("Attending")
#splitting("Zip")
splitting("Pat")

final_df[["BillingProviderID", "BillingProviderTIN", "BillingProviderOrg", "BillingProviderFName", "BillingProviderMName", "BillingProviderLName", "BillingProviderPhoneNumber", "BillingProviderAddressLine1", "BillingProviderAddressLine2", "BillingProviderCity","BillingProviderEntityType","BillingProviderNPI"]] = pd.DataFrame(final_df.pop('Billing').values.tolist(), index=final_df.index)
#final_df[["PrescribingProviderID", "PrescribingProviderTIN", "PrescribingProviderOrg", 'PrescribingProviderFName', 'PrescribingProviderMName', 'PrescribingProviderLName', 'PrescribingProviderPhoneNumber', "PrescribingProviderAddressLine1", "PrescribingProviderAddressLine2", 'PrescribingProviderCity',"PrescribingProviderNPI"]] = pd.DataFrame(final_df.pop('Prescribing').values.tolist(), index=final_df.index)
#final_df[["OrderingProviderID", 'OrderingProviderName',"OrderingProviderNPI"]] = pd.DataFrame(final_df.pop('Ordering').values.tolist(), index=final_df.index)
#final_df[['CertifyingPhysicianName', "CertifyingPhysicianID", "CertifyingPhysicianNPI"]] = pd.DataFrame(final_df.pop('Certifying').values.tolist(), index=final_df.index)
final_df[["ToothNumber","ToothBegin", "ToothEnd", "ToothSurfaces"]] = pd.DataFrame(final_df.pop('Tooth').values.tolist(), index=final_df.index)
final_df[["GroupName","GroupNumber"]] = pd.DataFrame(final_df.pop('Group').values.tolist(), index=final_df.index)
final_df[["AttendingProviderNAME","AttendingProviderNPI"]] = pd.DataFrame(final_df.pop('Attending').values.tolist(), index=final_df.index)
final_df[['PatientState', 'PatientFName', 'pat_gender', 'PatientLName', 'PatientMName', 'PatientID', 'PatientAddressLine1', 'PatientAddressLine2', 'PatientCity', 'SubscriberState', 'SubscriberID', 'PatientRelationship', 'PatientDOB', 'SubscriberGender', 'SubscriberDOB', 'SubscriberCity', 'SubscriberAddressLine1', 'SubscriberAddressLine2', 'SubscriberLName', 'SubscriberFName', 'SubscriberMName']] = pd.DataFrame(final_df.pop('Pat').values.tolist(), index=final_df.index)
#need to split up df based on age bin to do this correctly

#final_df[["PickUpZipCode","DropOffZipCode","BillingProviderZipCode",'RenderingProviderZipCode','PrescribingProviderZipCode','SubscriberZipCode','PatientZipCode','BillingProviderState',"RenderingProviderState",'SubscriberState','PrescribingProviderState']] = pd.DataFrame(final_df.pop('Zip').values.tolist(), index=final_df.index)
combo_list = final_df["combos"].tolist()

line_df = pd.DataFrame()
a = []
for index, row in final_df.iterrows():  
    b = []
    i = 0
    while i < len(row["combos"]):
        i+=1
        b.append(i)

    a.append(b)
line_df["LineNumber"] = a
line_df= line_df.explode('LineNumber')
"""
na_df = final_df.loc[(final_df['PickUpZipCode'] == '0')]
amb_df = final_df.loc[(final_df['PickUpZipCode'] != '0')]
na_df["APCCode"] = "Null"
amb_df["APCCode"] =np.random.randint(10000,99999, size=amb_df.shape[0])


final_df = pd.concat([amb_df, na_df])"""

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'\nna_df = final_df.loc[(final_df[\'PickUpZipCode\'] == \'0\')]\namb_df = final_df.loc[(final_df[\'PickUpZipCode\'] != \'0\')]\nna_df["APCCode"] = "Null"\namb_df["APCCode"] =np.random.randint(10000,99999, size=amb_df.shape[0])\n\n\nfinal_df = pd.concat([amb_df, na_df])'

In [11]:
final_df = final_df.explode('combos')#spliting up line into HCPCS and mod value in list, first is Hcpcs, second is mod for line
a = []
for index, row in final_df.iterrows():  
    b = (row['combos'].split(";"))
    a.append(b)
final_df["combos"] = a
final_df[['hcpcs_code','mod_values']] = pd.DataFrame(final_df.pop('combos').values.tolist(), index=final_df.index)

final_df['ren_id'] = final_df['ren_id'].apply(str)
rendering_df['ren_id'] = rendering_df['ren_id'].apply(str)
final_df = final_df.merge(rendering_df,on='ren_id')
final_df["combo"] = final_df["hcpcs_code"] +"@"+final_df["mod_values"]


cost = pd.read_csv("line_amt_df.csv", dtype={'place_of_service': str}, encoding= 'unicode_escape')
cost['hcpcs_code'] = cost['hcpcs_code'].astype(str)
cost['mod_values'] = cost['mod_values'].astype(str)
cost

a = []
for index, row in cost.iterrows():  
    b = (row['hcpcs_code'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)

cost["combo"] = a
cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])

c = cost.groupby(['combo'])['amount_charged'].sum()
c = c.reset_index()
e = cost.groupby(['combo'],as_index = False).size()
c["count"] = e["size"]
c["avg_charge"] = c["amount_charged"]/ c["count"]

ds = cost[cost.amount_paid !=0]
d = ds.groupby(['combo'])['amount_paid'].sum()
d = d.reset_index()
e = ds.groupby(['combo'], as_index= False).size()
d["count"] = e["size"]
d = d.merge(c[["combo", "avg_charge"]])
d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/d["avg_charge"]

f = cost.groupby(['combo'])['amount_paid'].sum()
f = f.reset_index()
e = cost.groupby(['combo'], as_index=False).size()
# e = e.reset_index()
f["count"] = e["size"]
f = f.merge(c[["combo", "avg_charge"]])
f["avg_paid_pct"] = (f["amount_paid"]/ f["count"])/f["avg_charge"]
f = f[f.avg_paid_pct == 0.000000]

d = d.append(f)


scb = 50      #may want to change scale so that it is more in line with reality
sca = .05

cost_dict = c.set_index('combo').to_dict()
final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
final_df['real charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
final_df['real charge'] = final_df['real charge'].abs()
final_df = final_df.drop(columns=['cost'])

paid_dict = d.set_index('combo').to_dict()
final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
final_df['real paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["real charge"].astype(int)
final_df['real paid'] = final_df['real paid'].abs()

final_df = final_df.drop(columns=['paid'])
final_df["real paid"] = final_df["real paid"].astype(int)



final_df["claim"] = final_df["prov_claim_id"]+"@"+final_df["ren_id"]
final_df_1 = final_df
final_df_1.groupby(['claim']).agg({'real charge': sum, 'real paid': sum}).reset_index()
final_df_1

final_df["total charged"] = final_df_1["real charge"]
final_df["total paid"] = final_df_1["real paid"]
final_df = final_df.drop(columns=['claim'])
final_df['Specialty_code'] = pd.Series(["25" for x in range(len(final_df.index))])

adult_df = final_df.loc[(final_df['age_bin'] == 'Adult')]
elderly_df = final_df.loc[(final_df['age_bin'] == 'Elderly')]
minor_df = final_df.loc[(final_df['age_bin'] == 'Minor')]

adult_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 22, maximum_age=65) for i in range(adult_df.shape[0])]
elderly_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 65, maximum_age=99) for i in range(elderly_df.shape[0])]
minor_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 1, maximum_age=18) for i in range(minor_df.shape[0])]

adult_df["PatientRelationship"] = np.random.choice(["Dependant", "Spouse", "Subscriber"], adult_df.shape[0], p=[0.2, 0.4, 0.4])
elderly_df["PatientRelationship"] = np.random.choice(["Spouse", "Subscriber"], elderly_df.shape[0], p=[0.5, 0.5])
minor_df["PatientRelationship"] = "Dependant"
final_df = pd.concat([adult_df, elderly_df, minor_df])

line = line_df["LineNumber"].tolist()

final_df["LineNumber"]= line



cost = pd.read_csv("line_amt_df.csv", dtype={'place_of_service': str}, encoding= 'unicode_escape')
cost['hcpcs_code'] = cost['hcpcs_code'].astype(str)
cost['mod_values'] = cost['mod_values'].astype(str)


a = []
for index, row in cost.iterrows():  
    b = (row['hcpcs_code'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)

cost["combo"] = a
cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])

c = cost.groupby(['combo'])['amount_charged'].sum()
c = c.reset_index()
e = cost.groupby(['combo'],as_index = False).size()
c["count"] = e["size"]
c["avg_charge"] = c["amount_charged"]/ c["count"]

ds = cost[cost.amount_paid !=0]
d = ds.groupby(['combo'])['amount_paid'].sum()
d = d.reset_index()
e = ds.groupby(['combo'], as_index= False).size()
d["count"] = e["size"]
d = d.merge(c[["combo", "avg_charge"]])
d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/d["avg_charge"]

f = cost.groupby(['combo'])['amount_paid'].sum()
f = f.reset_index()
e = cost.groupby(['combo'], as_index=False).size()
# e = e.reset_index()
f["count"] = e["size"]
f = f.merge(c[["combo", "avg_charge"]])
f["avg_paid_pct"] = (f["amount_paid"]/ f["count"])/f["avg_charge"]
f = f[f.avg_paid_pct == 0.000000]
d = d.append(f)

scb = 50      #may want to change scale so that it is more in line with reality
sca = .05
cost_dict = c.set_index('combo').to_dict()
final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
final_df['real charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
final_df['real charge'] = final_df['real charge'].abs()
final_df = final_df.drop(columns=['cost'])

paid_dict = d.set_index('combo').to_dict()
final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
final_df['real paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["real charge"].astype(int)
final_df['real paid'] = final_df['real paid'].abs()

final_df = final_df.drop(columns=['paid'])
final_df["real paid"] = final_df["real paid"].astype(int)

final_df["claim"] = final_df["prov_claim_id"]+"@"+final_df["ren_id"]
df_1 = final_df
df_1.groupby(['claim']).agg({'real charge': sum, 'real paid': sum}).reset_index()
df_1

final_df["total charged"] = df_1["real charge"]
final_df["total paid"] = df_1["real paid"]
final_df = final_df.drop(columns=['claim'])
final_df['Specialty_code'] = pd.Series(["25" for x in range(len(final_df.index))])

final_df


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,place_of_service,pat_gender,age_bin,diag_values,ren_id,prov_claim_id,ClaimID,CertifyingPhysicianName,CertifyingPhysicianID,CertifyingPhysicianNPI,...,RenderingProviderOrg,CCN,RenderingProviderNPI,combo,real charge,real paid,total charged,total paid,Specialty_code,LineNumber
19,22,Male,Adult,F4323,1,claim_0,1,Null,Null,Null,...,O,10701,75076,90837@95@WD,131,98,131,98,25,1
20,22,Male,Adult,F411,1,claim_1,2,Null,Null,Null,...,O,10701,75076,90837@95@WD,197,150,197,150,25,1
21,30,Female,Adult,F411,1,claim_2,3,Null,Null,Null,...,O,10701,75076,90837@WD,138,86,138,86,25,1
22,30,Female,Adult,F4320,1,claim_3,4,Null,Null,Null,...,O,10701,75076,90837@WD,131,73,131,73,25,1
23,30,Female,Adult,F4322,1,claim_4,5,Null,Null,Null,...,O,10701,75076,90791@WD,131,77,131,77,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138186,30,Female,Minor,F4323,1000,claim_106,134884,Null,Null,Null,...,O,24715,203470,90837@WD,139,77,139,77,25,1
138187,30,Female,Minor,F4320,1000,claim_112,134890,Null,Null,Null,...,O,24715,203470,90834@WD,119,64,119,64,25,1
138188,22,Male,Minor,F411,1000,claim_114,134892,Null,Null,Null,...,O,24715,203470,90837@GT@WD,191,130,191,130,25,1
138189,22,Female,Minor,F411,1000,claim_115,134893,Null,Null,Null,...,O,24715,203470,90837@95@WD,95,79,95,79,25,1


In [12]:
def splitting2(name, num):
    a = []
    for index, row in final_df.iterrows():  
        b = (row[name].split("@"))
        while len(b) < num:
            b.append("Null")
        a.append(b)
    final_df[name] = a


splitting2("mod_values",6)
splitting2("diag_values",13)

final_df[["MOD1","MOD2","MOD3","MOD4","MOD5","MOD6"]] = pd.DataFrame(final_df.pop('mod_values').values.tolist(), index=final_df.index)
final_df[["Diag1", "Diag2", "Diag3", "Diag4", "Diag5", "Diag6", "Diag7", "Diag8", "Diag9", "Diag10", "Diag11", "Diag12","Diag13"]] = pd.DataFrame(final_df.pop('diag_values').values.tolist(), index=final_df.index)

final_df.rename(columns = {'pat_gender':'PatientGender', "place_of_service": "LinePOS", "total paid":"LineTotPaid", "total charged":"LineCharges", "hcpcs_code":"LineProcCode"}, inplace = True)
final_df = final_df.drop(columns=["age_bin",'combo',"real charge","real paid", 'ren_id',"Specialty_code","prov_claim_id"])


final_df["LineTotPaid"] =final_df["LineTotPaid"].astype(int) 
nm_df = final_df.loc[(final_df['LineTotPaid'] <= 20)]
sm_df = final_df.loc[(final_df['LineTotPaid'] >= 20)]
nm_df["LineAmountCopay"] = nm_df["LineTotPaid"]

nm_df["LineAmountCoinsurance"] = 0
nm_df["LineAmountDeductible"] = 0
nm_df["LinePayerPaid"] = 0
sm_df["LineAmountCoinsurance"] = np.random.randint(0,20, size=sm_df.shape[0])/100
sm_df["LineAmountDeductible"]= np.random.randint(0,40, size=sm_df.shape[0])/100

sm_df["LineAmountCopay"]=[random.randrange(5,21,5)for i in range(sm_df.shape[0])]
sm_df["LinePayerPaid"]=sm_df["LineTotPaid"]-sm_df["LineAmountCopay"]
sm_df["LineAmountCoinsurance"] = sm_df["LineAmountCoinsurance"] *sm_df["LinePayerPaid"]//1
sm_df["LinePayerPaid"]=sm_df["LineTotPaid"]-sm_df["LineAmountCopay"]- sm_df["LineAmountCoinsurance"] 
sm_df["LineAmountDeductible"] = sm_df["LineAmountDeductible"]*sm_df["LinePayerPaid"]//1

final_df = pd.concat([sm_df, nm_df])
final_df["LinePatientPaid"] = final_df["LineAmountCopay"]+final_df["LineAmountDeductible"]+final_df["LineAmountCoinsurance"]
final_df["LineCOBPay"] = "0"
final_df["HdrCOBPay"] = "0"
final_df = final_df.sort_index(ascending=True)

hdr_df= pd.DataFrame()

hdr_df["ClaimID"] = final_df["ClaimID"]
hdr_df["LineTotPaid"]= final_df["LineTotPaid"]

hdr_df["LineAmountCopay"] = final_df["LineAmountCopay"]
hdr_df["LinePayerPaid"] = final_df["LinePayerPaid"]
hdr_df["LineCharges"] = final_df["LineCharges"]
hdr_df["LinePatientPaid"]=final_df["LinePatientPaid"]

#HdrPatientPay'
hdr_df = hdr_df.groupby(['ClaimID'], as_index=False).agg({'LineTotPaid': 'sum', 'LinePatientPaid': 'sum', "LineAmountCopay":'sum', "LinePayerPaid":'sum', "LineCharges":'sum'})
hdr_df.rename(columns = {'LineTotPaid':'HdrPay', "LinePayerPaid": "HdrPayerPay", "LineAmountCopay":"HdrPatientCopay", "LineCharges":"HdrCharges","LinePatientPaid":"HdrPatientPay"}, inplace = True)

final_df = pd.merge(final_df, hdr_df, on='ClaimID')
#final_df = final_df[["ClaimID", "CapEpcIndicator", "SubscriberID", "SubscriberFName", "SubscriberMName", "SubscriberLName", "SubscriberDOB", "SubscriberGender", "SubscriberAddressLine1", "SubscriberAddressLine2", "SubscriberCity", "SubscriberState", "SubscriberZipCode", "PatientID", "PatientRelationship", "PatientFName", "PatientMName", "PatientLName", "PatientDOB", "PatientGender", "PatientAddressLine1", "PatientAddressLine2", "PatientCity", "PatientState", "PatientZipCode", "GroupNumber", "GroupName", "BillingProviderID", "BillingProviderNPI", "BillingProviderTIN", "BillingProviderSpecialtyCode", "BillingProviderTaxonomy", "BillingProviderOrg", "BillingProviderFName", "BillingProviderMName", "BillingProviderLName", "BillingProviderEntityType", "BillingProviderAddressLine1", "BillingProviderAddressLine2", "BillingProviderCity", "BillingProviderState", "BillingProviderZipCode", "BillingProviderPhoneNumber", "CertifyingPhysicianID", "CertifyingPhysicianName", "CertifyingPhysicianNPI", "OrderingProviderID", "OrderingProviderName", "OrderingProviderNPI", "ReferringProviderNPI", "InterPlan", "LineOfBusiness", "ClaimPaidDate", "ClaimReceivedDate", "ClaimServiceEndDate", "ClaimServiceStartDate", "ClaimAdjustmentReasonCode", "PriorClaimID", "ASOFlag", "PrimaryFlag", "ClaimStatusCategoryCode", "ClaimType", "ContractedFlag", "HdrCharges", "HdrCOBPay", "HdrPatientCopay", "HdrPatientPay", "HdrPay", "HdrPayerPay", "CLIA", "DropOffZipCode", "PickUpZipCode", "RelatedCausesCode", "AccidentDate", "EmergencyIndicator", "LineNumber", "NumofAdjustments", "LinePatientPaid", "LineAmountCoinsurance", "LineAmountCopay", "LineAmountDeductible", "LineCharges", "LinePayerPaid", "LineCOBPay", "LineTotPaid", "LineProcCode", "LineUnits", "LineUnitType", "LineConsideredAmount", "LineAllowedAmount", "MOD1", "MOD2", "MOD3", "MOD4", "MOD5", "MOD6", "Diag1", "Diag2", "Diag3", "Diag4", "Diag5", "Diag6", "Diag7", "Diag8", "Diag9", "Diag10", "Diag11", "Diag12", "Diag13", "ICDVersion", "LinePOS", "LineServiceEndDate", "LineServiceStartDate", "LineServiceStatusCategoryCode", "LineAdjustmentReasonCode", "RenderingProviderID", "RenderingProviderNPI", "RenderingProviderTIN", "RenderingProviderSpecialtyCode", "RenderingProviderTaxonomy", "RenderingProviderOrg", "RenderingProviderFName", "RenderingProviderMName", "RenderingProviderLName", "RenderingProviderEntityType", "RenderingProviderAddressLine1", "RenderingProviderAddressLine2", "RenderingProviderCity", "RenderingProviderState", "RenderingProviderZipCode", "RenderingProviderPhoneNumber", "RelatedHospAdmitDate", "RelatedHospDischargeDate", "AdmitType", "AdmitSource", "DischargeStatus", "DRGCode", "DRGVersion", "AdmittingDiag", "ICDProc1", "ICDProc2", "ICDProc3", "ICDProc4", "ICDProcDate1", "ICDProcDate2", "ICDProcDate3", "ICDProcDate4", "APCCode", "CCN", "AttendingProviderNAME", "AttendingProviderNPI", "OtherPhysicianNAME", "OtherPhysicianNPI", "RevenueCode", "TypeofBill", "HCPC_Rate_HIPPSCode", "OccurrenceCode", "ServiceUnits", "PrescribingProviderID", "PrescribingProviderNPI", "PrescribingProviderTIN", "PrescribingProviderSpecialtyCode", "PrescribingProviderTaxonomy", "PrescribingProviderOrg", "PrescribingProviderFName", "PrescribingProviderMName", "PrescribingProviderLName", "PrescribingProviderAddressLine1", "PrescribingProviderAddressLine2", "PrescribingProviderCity", "PrescribingProviderState", "PrescribingProviderZipCode", "PrescribingProviderPhoneNumber", "OutOfNetwork", "PreAuthIndicator", "PreAuthNumber", "NDC", "DenyFlag", "DenyReason", "HIPPSCode", "AdmitDate", "ToothNumber", "ToothBegin", "ToothEnd", "ToothSurfaces", "ValueCode", "ConditionCode", "PayeeType"]]

final_df


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,LinePOS,PatientGender,ClaimID,CertifyingPhysicianName,CertifyingPhysicianID,CertifyingPhysicianNPI,OrderingProviderID,OrderingProviderName,OrderingProviderNPI,PrescribingProviderID,...,LineAmountCopay,LinePayerPaid,LinePatientPaid,LineCOBPay,HdrCOBPay,HdrPay,HdrPatientPay,HdrPatientCopay,HdrPayerPay,HdrCharges
0,22,Female,29,Null,Null,Null,Null,Frank,Null,Null,...,10,87.0,53.0,0,0,109,53.0,10,87.0,156
1,30,Female,31,Null,Null,Null,Null,Gibson,Null,Null,...,10,116.0,28.0,0,0,126,28.0,10,116.0,176
2,30,Male,38,Null,Null,Null,Null,Brown,Null,Null,...,20,108.0,31.0,0,0,130,31.0,20,108.0,173
3,30,Female,39,Null,Null,Null,Null,Sandoval,Null,Null,...,5,68.0,18.0,0,0,79,18.0,5,68.0,122
4,30,Male,53,Null,Null,Null,Null,Ortiz,Null,Null,...,4,0.0,4.0,0,0,4,4.0,4,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138450,30,Female,134897,Null,Null,Null,Null,Taylor,Null,Null,...,5,62.0,43.0,0,0,81,43.0,5,62.0,124
138451,30,Female,134898,Null,Null,Null,Null,Cox,Null,Null,...,10,73.0,32.0,0,0,84,32.0,10,73.0,127
138452,22,Female,134899,Null,Null,Null,Null,Reeves,Null,Null,...,10,118.0,28.0,0,0,130,28.0,10,118.0,186
138453,22,Male,134900,Null,Null,Null,Null,Hayes,Null,Null,...,5,119.0,41.0,0,0,125,41.0,5,119.0,166


In [13]:
final_df.to_csv("final_df.csv")

In [14]:
a = list(final_df.columns)
b = ["ClaimID", "CapEpcIndicator", "SubscriberID", "SubscriberFName", "SubscriberMName", "SubscriberLName", "SubscriberDOB", "SubscriberGender", "SubscriberAddressLine1", "SubscriberAddressLine2", "SubscriberCity", "SubscriberState", "SubscriberZipCode", "PatientID", "PatientRelationship", "PatientFName", "PatientMName", "PatientLName", "PatientDOB", "PatientGender", "PatientAddressLine1", "PatientAddressLine2", "PatientCity", "PatientState", "PatientZipCode", "GroupNumber", "GroupName", "BillingProviderID", "BillingProviderNPI", "BillingProviderTIN", "BillingProviderSpecialtyCode", "BillingProviderTaxonomy", "BillingProviderOrg", "BillingProviderFName", "BillingProviderMName", "BillingProviderLName", "BillingProviderEntityType", "BillingProviderAddressLine1", "BillingProviderAddressLine2", "BillingProviderCity", "BillingProviderState", "BillingProviderZipCode", "BillingProviderPhoneNumber", "CertifyingPhysicianID", "CertifyingPhysicianName", "CertifyingPhysicianNPI", "OrderingProviderID", "OrderingProviderName", "OrderingProviderNPI", "ReferringProviderNPI", "InterPlan", "LineOfBusiness", "ClaimPaidDate", "ClaimReceivedDate", "ClaimServiceEndDate", "ClaimServiceStartDate", "ClaimAdjustmentReasonCode", "PriorClaimID", "ASOFlag", "PrimaryFlag", "ClaimStatusCategoryCode", "ClaimType", "ContractedFlag", "HdrCharges", "HdrCOBPay", "HdrPatientCopay", "HdrPatientPay", "HdrPay", "HdrPayerPay", "CLIA", "DropOffZipCode", "PickUpZipCode", "RelatedCausesCode", "AccidentDate", "EmergencyIndicator", "LineNumber", "NumofAdjustments", "LinePatientPaid", "LineAmountCoinsurance", "LineAmountCopay", "LineAmountDeductible", "LineCharges", "LinePayerPaid", "LineCOBPay", "LineTotPaid", "LineProcCode", "LineUnits", "LineUnitType", "LineConsideredAmount", "LineAllowedAmount", "MOD1", "MOD2", "MOD3", "MOD4", "MOD5", "MOD6", "Diag1", "Diag2", "Diag3", "Diag4", "Diag5", "Diag6", "Diag7", "Diag8", "Diag9", "Diag10", "Diag11", "Diag12", "Diag13", "ICDVersion", "LinePOS", "LineServiceEndDate", "LineServiceStartDate", "LineServiceStatusCategoryCode", "LineAdjustmentReasonCode", "RenderingProviderID", "RenderingProviderNPI", "RenderingProviderTIN", "RenderingProviderSpecialtyCode", "RenderingProviderTaxonomy", "RenderingProviderOrg", "RenderingProviderFName", "RenderingProviderMName", "RenderingProviderLName", "RenderingProviderEntityType", "RenderingProviderAddressLine1", "RenderingProviderAddressLine2", "RenderingProviderCity", "RenderingProviderState", "RenderingProviderZipCode", "RenderingProviderPhoneNumber", "RelatedHospAdmitDate", "RelatedHospDischargeDate", "AdmitType", "AdmitSource", "DischargeStatus", "DRGCode", "DRGVersion", "AdmittingDiag", "ICDProc1", "ICDProc2", "ICDProc3", "ICDProc4", "ICDProcDate1", "ICDProcDate2", "ICDProcDate3", "ICDProcDate4", "APCCode", "CCN", "AttendingProviderNAME", "AttendingProviderNPI", "OtherPhysicianNAME", "OtherPhysicianNPI", "RevenueCode", "TypeofBill", "HCPC_Rate_HIPPSCode", "OccurrenceCode", "ServiceUnits", "PrescribingProviderID", "PrescribingProviderNPI", "PrescribingProviderTIN", "PrescribingProviderSpecialtyCode", "PrescribingProviderTaxonomy", "PrescribingProviderOrg", "PrescribingProviderFName", "PrescribingProviderMName", "PrescribingProviderLName", "PrescribingProviderAddressLine1", "PrescribingProviderAddressLine2", "PrescribingProviderCity", "PrescribingProviderState", "PrescribingProviderZipCode", "PrescribingProviderPhoneNumber", "OutOfNetwork", "PreAuthIndicator", "PreAuthNumber", "NDC", "DenyFlag", "DenyReason", "HIPPSCode", "AdmitDate", "ToothNumber", "ToothBegin", "ToothEnd", "ToothSurfaces", "ValueCode", "ConditionCode", "PayeeType"]

print("Problems", (set(a).difference(b)))
print("Missing columns", (set(b).difference(a)))



Problems set()
Missing columns {'BillingProviderZipCode', 'PickUpZipCode', 'PrescribingProviderZipCode', 'PrescribingProviderState', 'RenderingProviderState', 'APCCode', 'RenderingProviderZipCode', 'PatientZipCode', 'SubscriberZipCode', 'BillingProviderState', 'DropOffZipCode'}


In [15]:
print(len(b)-len(a))

11
